In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from PIL import Image

In [2]:
tf.__version__

'2.10.0'

In [3]:
import os

In [4]:
rootdir = "./dataCNN/"
labels = []
for file in os.listdir(rootdir):
    labels.append(file)
    

In [5]:
labels

['1', '2', '3', '4']

In [6]:
x , y = [] , []

In [7]:
for index , label in enumerate(labels):
    path = rootdir + label
    for file in os.listdir(path):
        img = Image.open(os.path.join(path , file) , "r")
        img = np.asarray(img)
        x.append(img / 255)
        y.append(index)

In [8]:
len(x)

702

In [9]:
x[0]

array([[[0.19607843, 0.18431373, 0.11764706],
        [0.21176471, 0.2       , 0.14117647],
        [0.2       , 0.18431373, 0.14901961],
        ...,
        [1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [0.99607843, 0.99607843, 0.99607843]],

       [[0.18039216, 0.17254902, 0.11372549],
        [0.18823529, 0.18039216, 0.12941176],
        [0.18823529, 0.17647059, 0.14117647],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843]],

       [[0.15294118, 0.16078431, 0.10980392],
        [0.16078431, 0.16862745, 0.11764706],
        [0.17647059, 0.17647059, 0.1372549 ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       ...,

       [[0.0745098 , 0.08235294, 0.07843137],
        [0.0745098 , 0.08235294, 0.07843137],
        [0.08235294, 0

In [10]:
x = np.asarray(x)
y = np.asarray(y)

In [11]:
x.shape

(702, 256, 256, 3)

In [12]:
from functools import partial

defaultcon3d = partial(keras.layers.Conv2D , kernel_size = 3 , activation = 'relu' , padding = "SAME") 
model = keras.models.Sequential([
    defaultcon3d(kernel_size = 7 , filters = 64 , input_shape = [256,256,3]),
    keras.layers.MaxPooling2D(pool_size = 2),
    defaultcon3d(filters = 64),
    defaultcon3d(filters = 64),
    keras.layers.MaxPooling2D(pool_size = 2),
    defaultcon3d(filters = 128),
    defaultcon3d(filters = 128),
    keras.layers.MaxPooling2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 150 , activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 100 , activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = len(labels) , activation = 'softmax')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       7

In [14]:
model.compile(metrics = ["accuracy"] , loss = "sparse_categorical_crossentropy" , optimizer = "adam")

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
xtrain ,  xtest ,ytrain,  ytest = train_test_split(x , y , test_size = 0.2)

In [17]:
type(xtrain)

numpy.ndarray

In [18]:
xtrain.shape

(561, 256, 256, 3)

In [19]:
checkpoint = keras.callbacks.ModelCheckpoint('modelCNN.h5')
earlyStop = keras.callbacks.EarlyStopping(patience = 4 , restore_best_weights = True , monitor = 'accuracy')

In [20]:
history = model.fit(xtrain , ytrain , epochs = 10, callbacks = [checkpoint, earlyStop])

Epoch 1/10
18/18 [==============================] - 110s 6s/step - loss: 1.2810 - accuracy: 0.4706
Epoch 2/10
18/18 [==============================] - 95s 5s/step - loss: 0.2883 - accuracy: 0.9109
Epoch 3/10
18/18 [==============================] - 96s 5s/step - loss: 0.0894 - accuracy: 0.9608
Epoch 4/10
18/18 [==============================] - 115s 6s/step - loss: 0.0447 - accuracy: 0.9911
Epoch 5/10
18/18 [==============================] - 111s 6s/step - loss: 0.0397 - accuracy: 0.9875
Epoch 6/10
18/18 [==============================] - 106s 6s/step - loss: 0.0276 - accuracy: 0.9893
Epoch 7/10
18/18 [==============================] - 134s 7s/step - loss: 0.0199 - accuracy: 0.9875
Epoch 8/10
18/18 [==============================] - 129s 7s/step - loss: 0.0300 - accuracy: 0.9911


In [21]:
history

In [22]:
preds = model.predict(xtest)

5/5 [==============================] - 8s 1s/step


In [23]:
type(preds)

numpy.ndarray

In [24]:
ans = []
for pred in preds:
    ans.append(np.argmax(pred))

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(ans , ytest)

array([[38,  0,  0,  0],
       [ 0, 37,  0,  0],
       [ 0,  0, 20,  0],
       [ 0,  0,  0, 46]], dtype=int64)

In [27]:
ans

[3,
 3,
 2,
 0,
 0,
 1,
 2,
 0,
 3,
 0,
 3,
 2,
 3,
 1,
 3,
 2,
 1,
 0,
 3,
 0,
 3,
 3,
 0,
 3,
 0,
 0,
 1,
 1,
 3,
 2,
 0,
 3,
 3,
 3,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 3,
 0,
 2,
 3,
 3,
 1,
 3,
 3,
 1,
 3,
 3,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 3,
 1,
 1,
 0,
 1,
 3,
 3,
 0,
 3,
 1,
 0,
 3,
 3,
 3,
 2,
 3,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 3,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 3,
 1,
 3,
 0,
 0,
 3,
 1,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 3,
 0,
 1,
 0,
 3,
 0,
 1,
 0,
 0,
 2,
 1,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 1,
 0,
 3,
 3,
 1,
 3,
 3,
 2,
 1,
 1,
 1,
 1,
 1]

In [28]:
preds

array([[3.53006913e-09, 7.78258613e-09, 1.17331672e-10, 1.00000000e+00],
       [5.66964864e-10, 6.23983865e-10, 3.01454955e-12, 1.00000000e+00],
       [1.54329116e-09, 1.92661553e-09, 1.00000000e+00, 3.31363548e-11],
       [1.00000000e+00, 2.03631487e-11, 9.47475293e-11, 4.38536180e-10],
       [1.00000000e+00, 2.01104289e-09, 1.05173532e-08, 1.34378464e-08],
       [8.31767641e-12, 1.00000000e+00, 1.02322456e-11, 2.59750399e-09],
       [1.56299662e-09, 1.86693017e-09, 1.00000000e+00, 3.20795682e-11],
       [1.00000000e+00, 7.39507761e-11, 2.66715178e-10, 2.02427071e-08],
       [1.50796041e-07, 1.69263288e-07, 1.64370295e-08, 9.99999762e-01],
       [1.00000000e+00, 2.68604444e-11, 1.78672091e-10, 8.23286384e-10],
       [1.47389978e-09, 1.23829182e-07, 5.80330284e-10, 9.99999881e-01],
       [1.48347234e-09, 1.77728987e-09, 1.00000000e+00, 3.00036523e-11],
       [6.34332586e-10, 5.54158719e-10, 4.22903639e-12, 1.00000000e+00],
       [6.85640910e-12, 1.00000000e+00, 1.11709678e